In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import ElasticNet
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np

from sklearn import metrics

import joblib

2024-02-16 07:20:20.291875: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 07:20:20.323091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 07:20:20.323124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 07:20:20.323978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 07:20:20.329170: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 07:20:20.330706: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [25]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [6]:
model = joblib.load("/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

2024-02-16 07:23:12.065254: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-16 07:23:12.424311: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://a6c02812a36e49eabe00ab0c90dfd5d9: INVALID_ARGUMENT: ram://a6c02812a36e49eabe00ab0c90dfd5d9 is a directory.


In [74]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [261]:
x_test_trimmed = np.concatenate((x_test[21500:21600], x_test[21900:22100], x_test[23800:23900], x_test[28000:28100], x_test[28300:28400]))
y_test_trimmed = np.concatenate((y_test[21500:21600], y_test[21900:22100], y_test[23800:23900], y_test[28000:28100], y_test[28300:28400]))
# 23800 23900 0 2 3 4 5 6 7 8 10 11 12 14 18(1 9 13 15 16 17)
# 21500 21600 0 3 4 6 7 8 9 10 11 12 14 18(1 13 15 16 17)
# 21900 22100 0 3 4 6 7 8 10 11 12 13 14 15 16 18(1 17)
# 28000 28100 0 3 4 6 7 8 11 12 14 15 17 18(17)
# 28300 28400 0 1 3 4 6 7 8 10 11 12 14 17 18

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=19,
    input_shape=(58,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [263]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.001)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 600/600 [2:06:38<00:00, 12.66s/it]  


In [264]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

19/19 [==============================] - 0s 671us/step
Accuracy: 0.03
AUC: 0.9060754860846234

macro
Precision: 0.09677286588307954
Recall: 0.1252297410192147
F1 Score: 0.062277172163854366

weighted
Precision: 0.008460521329910767
Recall: 0.03
F1 Score: 0.010622644297437995

Mean FNR: 0.8747702589807853
Mean TNR: 0.933447555683639
Mean FPR: 0.06655244431636113
Mean TPR: 0.1252297410192147


In [265]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 600/600 [2:06:38<00:00, 12.66s/it]  


In [267]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

19/19 [==============================] - 0s 659us/step
Accuracy: 0.025
AUC: 0.9240832656223263

macro
Precision: 0.09727830125473214
Recall: 0.11883876357560567
F1 Score: 0.06291004033935847

weighted
Precision: 0.007074964178059597
Recall: 0.025
F1 Score: 0.00855405168147704

Mean FNR: 0.8811612364243944
Mean TNR: 0.9329212176074063
Mean FPR: 0.06707878239259374
Mean TPR: 0.11883876357560567


In [268]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 600/600 [2:06:58<00:00, 12.70s/it]  


In [269]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

19/19 [==============================] - 0s 678us/step
Accuracy: 0.023333333333333334
AUC: 0.9184348826572794

macro
Precision: 0.16543459535757354
Recall: 0.11620718462823727
F1 Score: 0.0738058591949561

weighted
Precision: 0.009773367824587336
Recall: 0.023333333333333334
F1 Score: 0.009273994405457066

Mean FNR: 0.8837928153717627
Mean TNR: 0.9322807850229111
Mean FPR: 0.06771921497708906
Mean TPR: 0.11620718462823727
